In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from utils import *

In [2]:
train = pd.read_csv('data/train_ksmod.csv').fillna(0)
test = pd.read_csv('data/test_ksmod.csv').fillna(0)

In [3]:
sample = pd.read_csv('data/sample_submission.csv')

In [139]:
param = {'n_estimators': list(range(30, 80))}
rfc = RandomForestClassifier(n_jobs=-1)
gs = GridSearchCV(rfc, param)
gs.fit(
    train
    .iloc[
        :, 
        list(range(train_event_start, train_event_end)) +
        list(range(train_severity_start, train_severity_end)) +
        list(range(train_volume_start, train_end))], 
    train.loc[:, 'fault_severity'])

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'n_estimators': [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=0)

In [140]:
gs.best_params_

{'n_estimators': 72}

In [143]:
param = {'min_samples_split': list(range(2, 10))}
rfc = RandomForestClassifier(n_estimators=46, n_jobs=-1)
gs = GridSearchCV(rfc, param)
gs.fit(
    train
    .iloc[
        :, 
        list(range(train_event_start, train_event_end)) +
        list(range(train_severity_start, train_severity_end)) +
        list(range(train_volume_start, train_end))], 
    train.loc[:, 'fault_severity'])

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=46, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=0)

In [144]:
gs.best_params_

{'min_samples_split': 9}

In [5]:
rfc = RandomForestClassifier(n_estimators=72, min_samples_split=9, n_jobs=-1)

In [6]:
rfc.fit(
    train
    .iloc[
        :, 
        list(range(train_event_start, train_event_end)) +
        list(range(train_severity_start, train_severity_end)) +
        list(range(train_volume_start, train_end))], 
    train.loc[:, 'fault_severity'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=9,
            min_weight_fraction_leaf=0.0, n_estimators=72, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [7]:
prob = rfc.predict_proba(
    test
    .iloc[
        :, 
        list(range(test_event_start, test_event_end)) +
        list(range(test_severity_start, test_severity_end)) +
        list(range(test_volume_start, test_end))])

In [8]:
rfc_res = pd.DataFrame({
    'id': sample.loc[:, 'id'].values, 
    'predict_0': prob[:, 0],
    'predict_1': prob[:, 1],
    'predict_2': prob[:, 2]})

In [9]:
rfc_res.to_csv('rfc.csv', index=False)